# Tarea 3 - Reconocimiento de patrones en Minería de datos

## Integrantes

|Nombre | Rol|
|:------|:---|
|Felipe Avaria| 2923547-3|
|Rolando Casanueva| 201204505-3|


## Desarrollo

- Descripción de Apache Spark y MLlib

**Apache spark** es una herramienta rápida y genérica para el procesamientos de datos a gran escala *Big Data*. Provee una **API** de alto nivel en distintos lenguages, tales como *Java*, *Scala*, *Python* y *R* y además incluye un sistema optimizado para ejecuciones de gráficos. También soporta herramientas como SQL tanto en estructura como procesamiento, a la vez, posee MLlib para máquinas de aprendizaje, GraphX para procesamiento de gráficos y Spark Streaming. Por otra parte **MLlib** como se mencionó, está enfocado en máquinas de aprendizaje. Esta contiene una gran variedad de algoritmos para clasificación, regresión, recomendación, clustering entre otros. Y como en esta tarea realizaremos un sistema recomendador, podemos notar como la seccions de recomendación que incluye *ALS* (Alternating least squares) está presente, será nuestra guía. [Documentación PySpark MLlib](https://spark.apache.org/docs/latest/api/python/pyspark.mllib.html)



- ¿Qué son los RDD y Dataframe?

**RDD** (Resilient Distributed Dataset) es una abstracción básica de Spark, la cual representa una colección de elementos inmutables y particionados que puede ser operados de manera paralela. Mientras que **Dataframe**, es una colección de elementos agrupados bajo nombres de columnas. Este últimos se maneja mediantes un *SQLContext* dadas las caracteristicas del mismo. Ambos son agrupaciones de elementos pero sus estructuras los diferencian.


- Descripción del dataset utilizado.
